## Random Forest on Census Data

This code works on classifying a common UCI ML data set, Adults, which attempts to predict whether a person will make over 50k/year based on various characteristics (Age, race, sex, occupation, etc...)

It's a 1994 data set, so conclusions might not be super interesting to today's world, but it's good practice in implementing a random forest.

### Import data

In [2]:
import pandas as pd
import numpy as np

In [3]:
adults = pd.read_csv('adult.csv')
adults

,Age,workclass,FinalWeight,Education,EdNum,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


### Build a Decision Tree from the data

To build a decision tree, we need to decide which variable to put at the top, and how to split it below, on and on and on. 

We will use the Gini Impurtiy to find which factors are the best to incorporate!

For each column of interest, we will find the Gini impurity. For columns with numeric data or more than 2 options (most columns), we find the gini impurity for each individual value, then pick the lowest gini impurity from that batch to define the gini impurity of the column as a whole.

Let's find the gini impurity of two columns, one categorial and one numerical, to get an idea for the process. Then we'll write up functions to apply that process to other columns.

#### Age--Gini Impurity

The first step in finding the Gini Impurity for this column is ordering the data numerically and finding the mean value in between each value in the column.

In [14]:
#First, we create a numpy array from the dataset and sort it numerically. We also eliminate repeat values (of which there are many, of course)
age = np.unique(np.sort(np.array(adults['Age'])))

#We also print the array to make sure it makes sense.
print(age)

[17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64
 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88
 90]


We now need to find the average in between each age value. Just looking at this, it's clear it's just the 0.5 increments between the values, but we won't hard code it so it can apply to other variables as well. There might be a numpy package that can do this, but I think it's useful to write it out

In [30]:
age_avg_btwn = np.zeros(len(age) - 1)
for i in range(len(age) - 1):
    age_avg_btwn[i] = (age[i] + age[i+1])/2
print(age_avg_btwn)

[17.5 18.5 19.5 20.5 21.5 22.5 23.5 24.5 25.5 26.5 27.5 28.5 29.5 30.5
 31.5 32.5 33.5 34.5 35.5 36.5 37.5 38.5 39.5 40.5 41.5 42.5 43.5 44.5
 45.5 46.5 47.5 48.5 49.5 50.5 51.5 52.5 53.5 54.5 55.5 56.5 57.5 58.5
 59.5 60.5 61.5 62.5 63.5 64.5 65.5 66.5 67.5 68.5 69.5 70.5 71.5 72.5
 73.5 74.5 75.5 76.5 77.5 78.5 79.5 80.5 81.5 82.5 83.5 84.5 85.5 86.5
 87.5 89. ]


Perfect!

Now we need to calculate the gini inequality for each of these values. Essentially, we will split the data into people younger than and older than each age in this array, and then see how many people in each category make less than 50k or more than 50k. We find the lowest gini inequality for each age, and the age with the lowest will be the representative for the age column